# Threading

In [ ]:
from threading import Thread, Lock, current_thread
import time

In [ ]:
database_value = 0

def increase():
    global database_value
    local_copy = database_value
    
    #processing
    local_copy += 1
    time.sleep(0.1)
    database_value = local_copy
    
if __name__=="__main__":    
    print('start value', database_value)
    
    thread1 = Thread(target=increase)
    thread2 = Thread(target=increase)
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    print('end value', database_value)
    
    print('end main')
# we got 1 as an end value not 2 becasue thread2 appeal to the same variable (database_value) as thread1 

In [ ]:
database_value = 0

def increase(lock):
    global database_value
    lock.acquire()
    local_copy = database_value
    #processing
    local_copy += 1
    time.sleep(0.1)
    database_value = local_copy
    lock.release()
    
if __name__=="__main__":
    lock = Lock()
    
    print('start value', database_value)
    
    thread1 = Thread(target=increase, args=(lock,))
    thread2 = Thread(target=increase, args=(lock,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    print('end value', database_value)
    
    print('end main')
# we use lock to acquire a lock and then we have to release it to give acces for the second process and not get stuck
# now our end value is 2

In [ ]:
# instead of using lock.acquire() and lock.release() wwe can use with lock: and then code
database_value = 0

def increase(lock):
    global database_value
    with lock:
        local_copy = database_value
        local_copy += 1
        time.sleep(0.1)
        database_value = local_copy
    
if __name__=="__main__":
    lock = Lock()
    
    print('start value', database_value)
    
    thread1 = Thread(target=increase, args=(lock,))
    thread2 = Thread(target=increase, args=(lock,))
    
    thread1.start()
    thread2.start()
    
    thread1.join()
    thread2.join()
    
    print('end value', database_value)
    
    print('end main')

## Queue

In [ ]:
from queue import Queue

In [ ]:
if __name__=="__main__":
    q = Queue()
    
    q.put(1)
    q.put(2)
    q.put(3)
    # 3 2 1 -->
    first = q.get() # this will get and remove first item
    print(first) 
    q.task_done() # we always have to write it after everything is done
    q.join() # we block the main thread and wait until all the elements in queue are processed
    
    print('end main')

In [ ]:
# check if the queue is empty
q.empty()
# returns True if queue is empty or False if queue is not empty

In [ ]:
def worker(q, lock):
    while True:
        value = q.get()
        # processing...
        with lock:
            print(f'in {current_thread().name} got {value}')
        q.task_done()
        
if __name__=="__main__":
    q = Queue()
    lock = Lock()
    num_threads = 10
    
    for i in range(num_threads):
        thread = Thread(target=worker, args=(q,lock))
        thread.daemon = True # by default it is False
        thread.start()
    
    for i in range(1,21):
        q.put(i)
    q.join() # we have to block the main thread and wait until all the items are processed
        
    print('end main')
# our program ends because daemon thread dies when the main thread dies
# if we don't use a daemon thread then our program will still continue in while True loop